## Problema 2: Você deverá implementar um programa, para ler, tratar e particionar os dados.

O arquivo fonte está disponível em https://st-it-cloud-public.s3.amazonaws.com/people-v2_1E6.csv.gz



### Data Quality

- Higienizar e homogenizar o formato da coluna document
- Detectar através da coluna document se o registro é de uma Pessoa Física ou Pessoa Jurídica, adicionando uma coluna com essa informação
- Higienizar e homogenizar o formato da coluna birthDate

- Existem duas colunas nesse dataset que em alguns registros estão trocadas. Quais são essas colunas?
- Corrigir os dados com as colunas trocadas
- Além desses pontos, existem outras tratamentos para homogenizar esse dataset. Aplique todos que conseguir.


In [1]:
!pip install console-logging  
!pip install fuzzywuzzy
!sudo apt-get install libmysqlclient-dev --yes
!pip install dataset console-logging mysqlclient dataset --no-cache-dir



libmysqlclient-dev is already the newest version (8.0.27-0ubuntu0.20.04.1).


0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [2]:
from console_logging.console import Console
console = Console()
from datetime import datetime as dt
from Enriquecimento_UF import Logs

## Carregando Base 

In [3]:
import pandas as pd
import numpy as np
import re

etapa = "Etapa 2 - Data Clear"

console_log = Logs(etapa)

dataset = "https://st-it-cloud-public.s3.amazonaws.com/people-v2_1E6.csv.gz"


console_log.info("Loading dataset :" +dataset)


df = pd.read_csv(dataset,compression='gzip',error_bad_lines=False,sep=';',low_memory=False)

[19:25:1:872345][ INFO  ] : Etapa 2 - Data Clear | Loading dataset :https://st-it-cloud-public.s3.amazonaws.com/people-v2_1E6.csv.gz


/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


## Removendo caracter especial da coluna Document

In [4]:
console_log.info("Removento caracter especial")
df['document'] = df['document'].apply(lambda x: re.sub('[^A-Za-z0-9]+', '', x))

[19:27:7:925948][ INFO  ] : Etapa 2 - Data Clear | Removento caracter especial


In [5]:
df.head()

,document,name,job,jobArea,jobType,phoneNumber,birthDate,city,state,totalSpent
0,76684148787,Charlleny Braga,Oficial Criativo Dinâmico,Configuração,Estrategista,(62) 4216-9799,20-05-1972,Município de Iara,Goiás,913.80
1,85704855733,Newton Saraiva,Administrador Comunicações Internacional,Prestação de contas,Facilitador,Aplicações,10-Jun-1982,Município de Neide do Sul,RR,57.26
2,15664328373377,Dr. Sr. Solange Macedo,Designer Identidade Direto,Métricas,NaN,+55 (95) 7143-3307,05/16/1968,Município de Santo,RO,660.71
3,02328238087786,Celina Carvalho Jr.,NaN,Qualidade,Ligação,+55 (58) 4136-5577,"19810417,",Reis do Norte,RO,542.51
4,30073687408740,Aurilo Martins,Especialista Paradigma Internacional,Programa,Executivo,(96) 47498-7325,04/07/1980,Município de Aminadab,RO,104.33


## Avaliando Doc para Pessoas Juridica ou Fisica

In [6]:

console_log.info("Criando Coluna Pessoas, Pessoa fisica x Pessoa Juridica")
df['PESSOA'] = df['document'].apply(lambda document: "JURIDICA" if len(str(document)) == 14 else "FISICA")

[19:27:9:655657][ INFO  ] : Etapa 2 - Data Clear | Criando Coluna Pessoas, Pessoa fisica x Pessoa Juridica


## Correção e Parse de Data Hora

In [7]:
from dateutil.parser import parse

console_log.info("CORREÇÃO DATA HORA")

console.info("INICIO CORREÇÃO DATA HORA")
inicio = dt.now()
df['birthDate'] = df['birthDate'].apply(lambda date: parse (date))

console.info("FIM CORREÇÃO HORA")

console.log("Tempo de Exec: "+str(dt.now() - inicio))

[19:27:10:210415][ INFO  ] : Etapa 2 - Data Clear | CORREÇÃO DATA HORA
[19:27:10:215929][ INFO  ] : INICIO CORREÇÃO DATA HORA


[19:27:55:53003][ INFO  ] : FIM CORREÇÃO HORA
[19:27:55:53151][  LOG  ] : Tempo de Exec: 0:00:44.837098


## Processo de correção de phoneNumber e Job Area

In [8]:
df['phoneNumber'] = df['phoneNumber'].apply(lambda x: re.sub('[^A-Za-z0-9]+', '', x))

In [9]:
df['jobArea'] = df['jobArea'].apply(lambda x: x.replace("+","").replace("(","").replace(")","").replace("-","").replace(" ",""))

In [10]:
df.iloc[[20,21,22,42,44,45,46,49,63,70]]

,document,name,job,jobArea,jobType,phoneNumber,birthDate,city,state,totalSpent,PESSOA
20,30608514374605,Vanilda Carvalho,Supervisor Paradigma Regional,Operações,Analista,5281574122,1979-04-11,Grande Hernandes de Nossa Senhora,AM,722.46,JURIDICA
21,65270593106,Flormira Costa Neto,Designer Prestação de contas Chefe,Mercados,NaN,44137571479,1972-11-14,Vila Tochie do Descoberto,PI,617.65,FISICA
22,35171180369,Vanoil Xavier Filho Filho,Analista Comunicações Corporativo,554729539641,Analista,Operaes,1979-09-25,Velha Fátima do Descoberto,Paraná,402.10,FISICA
42,27511213308,Leila Melo,Ligação Resposta Frente,553538379102,Analista,3978491494,1965-06-05,Deise do Sul,Tocantins,521.93,FISICA
44,18273161285,Sinezio Saraiva Filho,Produtor Fatores Diretor,71492839575,Associado,4067855749,1980-01-11,Angelino do Norte,Rio de Janeiro,195.40,FISICA
45,18091073500,Mauricio Costa,NaN,Implementação,Representante,551545325212,1970-11-05,NaN,Amapá,514.80,FISICA
46,28673547067,Dr. Laura Nogueira Filho,Ligação Operações Global,Configuração,Coordenador,556813989342,1977-10-23,Costa de Nossa Senhora,Espírito Santo,275.79,FISICA
49,91131730810,Rizia Carvalho,Assistente Integração Frente,554500855603,Oficial,2068756735,1985-09-23,NaN,Roraima,717.84,FISICA
63,27534718198,Srta. Emerson Neto,Técnico Aplicações Internacional,16392243386,NaN,555744493385,1970-11-19,NaN,AC,615.94,FISICA
70,39481762483,Dr. Valdirene Costa Neto,Executivo Marca Regional,40953777274,Produtor,08994475247,1972-03-30,Grande Jurema do Descoberto,Ceará,863.68,FISICA


In [11]:
def fuck_corrige(df):
    aux = df.copy()

    for ids,item in df[['jobArea','phoneNumber']].iterrows():
        if item['jobArea'].isnumeric() == True and item['phoneNumber'].isnumeric() == False:
              aux.loc[ids,"jobArea"] = item["phoneNumber"]
              aux.loc[ids,"phoneNumber"]= item ["jobArea"]

        if item["jobArea"].isnumeric() == False and item['phoneNumber'].isnumeric() == True:
            pass    
        
        if item["jobArea"].isnumeric() == True and item['phoneNumber'].isnumeric() == True:
            aux.loc[ids,"jobArea"] =  ""

        if item["jobArea"].isnumeric() == False and item['phoneNumber'].isnumeric() == False:
            aux.loc[ids,"phoneNumber"]= "" 
    return aux 


In [12]:

inicio = dt.now()
console_log.info(" TRATAMENTO DE COLUNAS ")

df_aux = fuck_corrige(df)

console.log("Tempo de Exec: "+str(dt.now() - inicio))

[19:27:58:487254][ INFO  ] : Etapa 2 - Data Clear |  TRATAMENTO DE COLUNAS 


[20:5:42:807957][  LOG  ] : Tempo de Exec: 0:37:44.320710


In [13]:
df.iloc[[20,21,22,42,44,45,46,49,63,70]]

,document,name,job,jobArea,jobType,phoneNumber,birthDate,city,state,totalSpent,PESSOA
20,30608514374605,Vanilda Carvalho,Supervisor Paradigma Regional,Operações,Analista,5281574122,1979-04-11,Grande Hernandes de Nossa Senhora,AM,722.46,JURIDICA
21,65270593106,Flormira Costa Neto,Designer Prestação de contas Chefe,Mercados,NaN,44137571479,1972-11-14,Vila Tochie do Descoberto,PI,617.65,FISICA
22,35171180369,Vanoil Xavier Filho Filho,Analista Comunicações Corporativo,554729539641,Analista,Operaes,1979-09-25,Velha Fátima do Descoberto,Paraná,402.10,FISICA
42,27511213308,Leila Melo,Ligação Resposta Frente,553538379102,Analista,3978491494,1965-06-05,Deise do Sul,Tocantins,521.93,FISICA
44,18273161285,Sinezio Saraiva Filho,Produtor Fatores Diretor,71492839575,Associado,4067855749,1980-01-11,Angelino do Norte,Rio de Janeiro,195.40,FISICA
45,18091073500,Mauricio Costa,NaN,Implementação,Representante,551545325212,1970-11-05,NaN,Amapá,514.80,FISICA
46,28673547067,Dr. Laura Nogueira Filho,Ligação Operações Global,Configuração,Coordenador,556813989342,1977-10-23,Costa de Nossa Senhora,Espírito Santo,275.79,FISICA
49,91131730810,Rizia Carvalho,Assistente Integração Frente,554500855603,Oficial,2068756735,1985-09-23,NaN,Roraima,717.84,FISICA
63,27534718198,Srta. Emerson Neto,Técnico Aplicações Internacional,16392243386,NaN,555744493385,1970-11-19,NaN,AC,615.94,FISICA
70,39481762483,Dr. Valdirene Costa Neto,Executivo Marca Regional,40953777274,Produtor,08994475247,1972-03-30,Grande Jurema do Descoberto,Ceará,863.68,FISICA


In [14]:
df_aux.iloc[[20,21,22,42,44,45,46,49,63,70]]

,document,name,job,jobArea,jobType,phoneNumber,birthDate,city,state,totalSpent,PESSOA
20,30608514374605,Vanilda Carvalho,Supervisor Paradigma Regional,Operações,Analista,5281574122,1979-04-11,Grande Hernandes de Nossa Senhora,AM,722.46,JURIDICA
21,65270593106,Flormira Costa Neto,Designer Prestação de contas Chefe,Mercados,NaN,44137571479,1972-11-14,Vila Tochie do Descoberto,PI,617.65,FISICA
22,35171180369,Vanoil Xavier Filho Filho,Analista Comunicações Corporativo,Operaes,Analista,554729539641,1979-09-25,Velha Fátima do Descoberto,Paraná,402.10,FISICA
42,27511213308,Leila Melo,Ligação Resposta Frente,,Analista,3978491494,1965-06-05,Deise do Sul,Tocantins,521.93,FISICA
44,18273161285,Sinezio Saraiva Filho,Produtor Fatores Diretor,,Associado,4067855749,1980-01-11,Angelino do Norte,Rio de Janeiro,195.40,FISICA
45,18091073500,Mauricio Costa,NaN,Implementação,Representante,551545325212,1970-11-05,NaN,Amapá,514.80,FISICA
46,28673547067,Dr. Laura Nogueira Filho,Ligação Operações Global,Configuração,Coordenador,556813989342,1977-10-23,Costa de Nossa Senhora,Espírito Santo,275.79,FISICA
49,91131730810,Rizia Carvalho,Assistente Integração Frente,,Oficial,2068756735,1985-09-23,NaN,Roraima,717.84,FISICA
63,27534718198,Srta. Emerson Neto,Técnico Aplicações Internacional,,NaN,555744493385,1970-11-19,NaN,AC,615.94,FISICA
70,39481762483,Dr. Valdirene Costa Neto,Executivo Marca Regional,,Produtor,08994475247,1972-03-30,Grande Jurema do Descoberto,Ceará,863.68,FISICA


## Correção de UF 

In [15]:
ufs = pd.read_csv("https://raw.githubusercontent.com/mapaslivres/municipios-br/main/tabelas/ufs.csv")

In [16]:
!pip install fuzzywuzzy

In [17]:
import string
from fuzzywuzzy import fuzz

etapa = "Etapa 2 - Data Clear"

console_log = Logs(etapa)

console_log.info(" CORRIGINDO UF ")

lista_state = df_aux[['state']].groupby('state').count().index.tolist()

for index, value in ufs[['uf_code','name']].iterrows():
    #
    for uf in lista_state:
        distance = fuzz.ratio(value['name'], string.capwords(uf))  
        if distance > 60:
          print(value['uf_code'], value['name'],uf, distance)
          query = "state == '{}'".format(uf)
          print(query)  
          lista_index_estados = df_aux.query(query).index.to_list()
          df_aux.loc[lista_index_estados,'state'] = value['uf_code']  
          

/opt/conda/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[20:5:45:881226][ INFO  ] : Etapa 2 - Data Clear |  CORRIGINDO UF 
AC Acre AC 67
state == 'AC'
AC Acre Acre 100
state == 'Acre'


AL Alagoas Alagoas 100
state == 'Alagoas'
AL Alagoas Amazonas 67
state == 'Amazonas'
AM Amazonas Alagoas 67
state == 'Alagoas'
AM Amazonas Amazonas 100
state == 'Amazonas'


AP Amapá Amapá 100
state == 'Amapá'
BA Bahia Bahia 100
state == 'Bahia'
CE Ceará Ceará 100
state == 'Ceará'
CE Ceará Pará 67
state == 'Pará'
DF Distrito Federal Distrito Federal 100
state == 'Distrito Federal'


ES Espírito Santo Espírito Santo 100
state == 'Espírito Santo'
GO Goiás Goiás 100
state == 'Goiás'
MA Maranhão Maranhão 100
state == 'Maranhão'
MG Minas Gerais Minas Gerais 100
state == 'Minas Gerais'
MS Mato Grosso do Sul Mato Grosso 76
state == 'Mato Grosso'


...

PA Pará PA 67
state == 'PA'
PA Pará PR 67
state == 'PR'
PA Pará Paraná 80
state == 'Paraná'
PA Pará Pará 100
state == 'Pará'
PB Paraíba Paraná 62
state == 'Paraná'


PB Paraíba Paraíba 100
state == 'Paraíba'
PE Pernambuco Pernambuco 100
state == 'Pernambuco'
PI Piauí Piauí 100
state == 'Piauí'
PR Paraná Paraná 100
state == 'Paraná'
PR Paraná Paraíba 62
state == 'Paraíba'


PR Paraná Pará 80
state == 'Pará'
RJ Rio de Janeiro Rio de Janeiro 93
state == 'Rio de Janeiro'
RN Rio Grande do Norte Rio Grande do Norte 95
state == 'Rio Grande do Norte'
RN Rio Grande do Norte Rio Grande do Sul 72
state == 'Rio Grande do Sul'
RO Rondônia Rondônia 100
state == 'Rondônia'


RR Roraima Roraima 100
state == 'Roraima'
RS Rio Grande do Sul Rio Grande do Norte 72
state == 'Rio Grande do Norte'
RS Rio Grande do Sul Rio Grande do Sul 94
state == 'Rio Grande do Sul'
SC Santa Catarina Santa Catarina 100
state == 'Santa Catarina'
SE Sergipe Sergipe 100
state == 'Sergipe'


SP São Paulo São Paulo 100
state == 'São Paulo'
TO Tocantins Tocantins 100
state == 'Tocantins'


In [18]:
df_aux

,document,name,job,jobArea,jobType,phoneNumber,birthDate,city,state,totalSpent,PESSOA
0,76684148787,Charlleny Braga,Oficial Criativo Dinâmico,Configuração,Estrategista,6242169799,1972-05-20,Município de Iara,GO,913.80,FISICA
1,85704855733,Newton Saraiva,Administrador Comunicações Internacional,Prestaçãodecontas,Facilitador,,1982-06-10,Município de Neide do Sul,RR,57.26,FISICA
2,15664328373377,Dr. Sr. Solange Macedo,Designer Identidade Direto,Métricas,NaN,559571433307,1968-05-16,Município de Santo,RO,660.71,JURIDICA
3,02328238087786,Celina Carvalho Jr.,NaN,Qualidade,Ligação,555841365577,1981-04-17,Reis do Norte,RO,542.51,JURIDICA
4,30073687408740,Aurilo Martins,Especialista Paradigma Internacional,Programa,Executivo,96474987325,1980-04-07,Município de Aminadab,RO,104.33,JURIDICA
...,...,...,...,...,...,...,...,...,...,...,...
999995,20297054201,Aurilo Franco Neto,Agente Funcionalidade Investidor,Otimização,NaN,5487425107,1976-04-16,Vila Attilio,RO,180.85,FISICA
999996,76245248035484,Iata Martins,Técnico A infraestrutura Central,Rede,NaN,7280660110,1981-05-22,Grande Roverlei,PB,89.75,JURIDICA
999997,15787105200206,Erica Melo,Associado Operações Cliente,Garantia,NaN,0053162453,1988-01-23,NaN,MA,138.48,JURIDICA
999998,12922681661,Jurema Carvalho Filho Neto,Agente Operações Legado,Programa,Engenheiro,554486912473,1968-04-11,NaN,RN,827.67,FISICA


## Salvando dados 

In [19]:
!mkdir base_gerada

In [20]:
console_log.info(" TRATANDO ANO  ")

[20:5:49:138521][ INFO  ] : Etapa 2 - Data Clear |  TRATANDO ANO  


In [21]:
df_aux["Ano"] = df_aux["birthDate"].apply(lambda x:str(x).split("-")[0])

In [22]:
lista_anos = df_aux[['Ano']].groupby('Ano').count().index.tolist()

In [23]:
[df_aux.query('Ano == "{}"'.format(ano)).to_csv("base_gerada/{}.csv".format(ano),index=False)  for ano in lista_anos ]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]


### Particionamento de dados tratados com as regras descritas em DATA QUALITY
- Particionar em arquivos PARQUET por estado (state)
- Particionar em arquivos CSV por ano/mes/dia de nascimento (birthDate)

In [24]:
!pip install fastparquet

     |███                             | 112 kB 5.3 MB/s eta 0:00:01

     |██████████▉                     | 419 kB 5.3 MB/s eta 0:00:01

     |███████████████████             | 737 kB 5.3 MB/s eta 0:00:01

     |██████████████████████████▍     | 1.0 MB 5.3 MB/s eta 0:00:01

     |████████████████████████████████| 1.2 MB 5.3 MB/s            


...

 |

 /

 -

 \

 done
  Created wheel for thrift: filename=thrift-0.15.0-cp39-cp39-linux_x86_64.whl size=183119 sha256=786ed2433b3ca9d16d427748c8c9b8a191cf983cfa3a290a7ee5eeac005a9785
  Stored in directory: /tmp/pip-ephem-wheel-cache-sxdpq28g/wheels/00/41/cd/f65d329f7cc147ce79cc833b11b94f7bea7329df0ad9d81629
Successfully built thrift


In [25]:
df_aux.to_parquet('base_gerada/df.parquet.gzip',compression='gzip')

In [26]:
del df
del df_aux

In [27]:
console_log.info(" FIM ETAPA 2 - NOTEBOOK  ")

[20:6:21:567968][ INFO  ] : Etapa 2 - Data Clear |  FIM ETAPA 2 - NOTEBOOK  



### Agregação dos dados Essa resposta estara no (BI) Metabase
- Quais são as 5 PF que mais gastaram (totalSpent)?
- Qual é o valor de gasto médio por estado (state)?
- Qual é o valor de gasto médio por jobArea?
- Qual é a PF que gastou menos (totalSpent)?
- Quantos nomes e documentos repetidos existem nesse dataset?
- Quantas linhas existem nesse dataset?

